# AI Config Context (Basic) - Cookbook

This cookbook tests all code from `aiconfig-context-basic/SKILL.md`.

## Prerequisites
- `LAUNCHDARKLY_SDK_KEY` in `.env` at repo root

In [1]:
# Install and load environment
%pip install launchdarkly-server-sdk launchdarkly-server-sdk-ai python-dotenv -q

import os
from pathlib import Path
from dotenv import load_dotenv

def find_repo_root(start_path: Path = None) -> Path:
    current = start_path or Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / '.git').exists():
            return parent
    return current

repo_root = find_repo_root()
load_dotenv(repo_root / '.env')
print(f"[OK] Loaded environment")

# Initialize SDK (for cleanup flush)
import ldclient
from ldclient.config import Config

SDK_KEY = os.environ.get("LAUNCHDARKLY_SDK_KEY")
ldclient.set_config(Config(SDK_KEY))
ld_client = ldclient.get()
print(f"[OK] SDK initialized")


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[OK] Loaded environment
[OK] SDK initialized


---
## Simple Context Creation
From: `SKILL.md` lines 32-46

In [2]:
from ldclient import Context

def create_basic_context(user_id: str):
    """Create a simple context with just user ID"""
    context = Context.builder(user_id).build()
    return context

# Test
context = create_basic_context("user-123")
print(f"[OK] create_basic_context(): key={context.key}, valid={context.valid}")

[OK] create_basic_context(): key=user-123, valid=True


---
## Context with Common Attributes
From: `SKILL.md` lines 48-82

In [3]:
def create_user_context(
    user_id: str,
    email: str = None,
    name: str = None,
    tier: str = None,
    country: str = None
):
    """Create context with common user attributes"""
    builder = Context.builder(user_id)
    if email:
        builder.set("email", email)
    if name:
        builder.set("name", name)
    if tier:
        builder.set("tier", tier)
    if country:
        builder.set("country", country)
    return builder.build()

# Test
context = create_user_context("user-123", name="Anna", tier="premium", country="US")
print(f"[OK] create_user_context(): name={context.get('name')}, tier={context.get('tier')}")

[OK] create_user_context(): name=Anna, tier=premium


---
## Context for Business Applications
From: `SKILL.md` lines 84-110

In [4]:
def create_business_context(user_id: str, user_data: dict, org_data: dict):
    """Create context for B2B applications"""
    builder = Context.builder(user_id)
    builder.set("role", user_data.get("role"))
    builder.set("department", user_data.get("department"))
    builder.set("companyName", org_data.get("name"))
    builder.set("companySize", org_data.get("size"))
    builder.set("industry", org_data.get("industry"))
    builder.set("plan", org_data.get("plan"))
    return builder.build()

# Test
context = create_business_context(
    "user-123",
    user_data={"role": "developer", "department": "engineering"},
    org_data={"name": "TechCorp", "size": "enterprise", "industry": "software", "plan": "premium"}
)
print(f"[OK] create_business_context(): role={context.get('role')}, company={context.get('companyName')}")

[OK] create_business_context(): role=developer, company=TechCorp


---
## Multi-Context for Complex Scenarios
From: `SKILL.md` lines 112-146

In [5]:
def create_multi_context(user_id: str, org_id: str, user_attrs: dict, org_attrs: dict):
    """Create multi-context for user within organization"""
    user_context = Context.builder(user_id).kind("user")
    for key, value in user_attrs.items():
        user_context.set(key, value)

    org_context = Context.builder(org_id).kind("organization")
    for key, value in org_attrs.items():
        org_context.set(key, value)

    multi_context = Context.multi_builder() \
        .add(user_context.build()) \
        .add(org_context.build()) \
        .build()
    return multi_context

# Test
context = create_multi_context(
    "user-123", "org-456",
    user_attrs={"role": "developer"},
    org_attrs={"plan": "enterprise"}
)
print(f"[OK] create_multi_context(): valid={context.valid}, multiple={context.multiple}")

[OK] create_multi_context(): valid=True, multiple=True


---
## Date and Time Attributes
From: `SKILL.md` lines 148-171

In [6]:
from datetime import datetime

def add_date_attributes(builder, signup_date):
    """Add date attributes in supported formats"""
    timestamp_ms = int(signup_date.timestamp() * 1000)
    builder.set("signupTimestamp", timestamp_ms)
    rfc3339_date = signup_date.isoformat() + "Z"
    builder.set("signupDate", rfc3339_date)
    days_since = (datetime.now() - signup_date).days
    builder.set("accountAgeDays", days_since)
    return builder

# Test
builder = Context.builder("user-123")
add_date_attributes(builder, datetime(2024, 1, 15))
context = builder.build()
print(f"[OK] add_date_attributes(): accountAgeDays={context.get('accountAgeDays')}")

[OK] add_date_attributes(): accountAgeDays=750


---
## Anonymous Contexts
From: `SKILL.md` lines 173-202

In [7]:
import uuid

def create_anonymous_context(session_id: str = None, **attributes):
    """Create context for anonymous users"""
    if not session_id:
        session_id = str(uuid.uuid4())
    builder = Context.builder(session_id)
    builder.anonymous(True)
    for key, value in attributes.items():
        builder.set(key, value)
    return builder.build()

# Test
context = create_anonymous_context(referrer="google", deviceType="mobile")
print(f"[OK] create_anonymous_context(): anonymous={context.anonymous}, referrer={context.get('referrer')}")

[OK] create_anonymous_context(): anonymous=True, referrer=google


---
## Best Practices
From: `SKILL.md` lines 212-258

---
## Common Use Cases for AI Configs
From: `SKILL.md` lines 260-302

---
## Common Use Cases for AI Configs
From: `SKILL.md` lines 287-329

In [8]:
# E-commerce AI Shopping Assistant
# tier: model selection, name: personalization
ecom_context = Context.builder("user-123") \
    .set("tier", "premium") \
    .set("name", "Sarah") \
    .set("preferredCategory", "electronics") \
    .build()
print(f"[OK] E-commerce AI: tier={ecom_context.get('tier')}")

# SaaS AI Copilot
# plan: model selection, aiCreditsRemaining: usage limits
saas_context = Context.builder("user-123") \
    .set("plan", "enterprise") \
    .set("aiCreditsRemaining", 500) \
    .set("role", "developer") \
    .build()
print(f"[OK] SaaS AI Copilot: plan={saas_context.get('plan')}")

# Content Platform AI Writer
# writingStyle/targetAudience: prompt customization
content_context = Context.builder("user-123") \
    .set("writingStyle", "technical") \
    .set("targetAudience", "expert") \
    .set("language", "en") \
    .build()
print(f"[OK] Content AI Writer: style={content_context.get('writingStyle')}")

[OK] E-commerce AI: tier=premium
[OK] SaaS AI Copilot: plan=enterprise
[OK] Content AI Writer: style=technical
